1. lucas0_train.csv 파일로 lung_cancer 타겟으로 하셔서 분류모형
- 파티셔닝
- DT, RF, GBC 모델링 각각
- 성능 비교
-제일 성능 좋은 모형에 대해서 GridSearch











In [ ]:
import pandas as pd
df = pd.read_csv("lucas0_train.csv")
df.head()

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("Lung_cancer", axis=1),
                                                    df.Lung_cancer, test_size=0.2,
                                                    stratify=df.Lung_cancer,
                                                    random_state=1)
models = [ DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier()]
for i in models:
  i.fit(X_train, y_train)
  print(i.score(X_test, y_test))

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(
    estimator=GradientBoostingClassifier(),
    param_grid = {"n_estimators":[50,100],
        "max_depth":[5,7],
        "learning_rate":[0.01, 0.005]
    },
    cv=3,
    verbose=10
)

gs.fit( X_train, y_train)

2. oil.csv
- 현재 일별 유가  -> 시계열
- 월별 유가 -> 시계열
- prophet
- 10건의 예측값을 계산

In [23]:
df = pd.read_csv("oil.csv", parse_dates=[0], index_col=[0])
df.dropna( inplace=True)
df.resample('M').last()   #월별
df  #일별 시계열
df1 = pd.DataFrame({"y":df.dcoilwtico, "ds": df.index })
df.reset_index()

from prophet import Prophet
model = Prophet()
model.fit(df1)

future = model.make_future_dataframe(periods=10)
forecast = model.predict(future)
forecast.yhat

/tmp/ipython-input-2915021084.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df.resample('M').last()   #월별
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqht4klw_/_2yhm3_0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqht4klw_/5pc8ovum.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=72944', 'data', 'file=/tmp/tmpqht4klw_/_2yhm3_0.json', 'init=/tmp/tmpqht4klw_/5pc8ovum.json', 'output', 'file=/tmp/tmpqht4klw_/prophet_model65zwbeip/prophet_model-20250806065534.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
06:55:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
06:55:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:C

,yhat
0,92.669127
1,92.551483
2,92.330600
3,91.777130
4,91.725877
...,...
1180,49.284434
1181,49.358632
1182,49.311755
1183,46.799411




3. mortdefault.zip ->2007, 2008, 2009
- 2007 default레코드, 같은 수의 정상레코드를
- sample->하나의 df1, year컬럼->2007
- df1["year"] = "2007"

- 2008년도, 2009년도 동일하게 처리
 - 세 개의 df를 하나로 합치세요 pd.concat( [df1, df2, df3]     )

- creditscore, ....., default, year
- ......................................., 2007
- ........................................, 2008

- year를 OHE / target: default/파티셔닝, DT, RF
- feature_importances_


In [25]:
!unzip mortDefault.zip

Archive:  mortDefault.zip
  inflating: mortDefault2007.csv     
  inflating: mortDefault2008.csv     
  inflating: mortDefault2009.csv     


In [26]:
df1 = pd.read_csv("mortDefault2007.csv")
df2 = pd.read_csv("mortDefault2008.csv")
df3 = pd.read_csv("mortDefault2009.csv")

In [31]:
df2007 = pd.concat([ df1.query( "default==1"),
                    df1.query( "default==0").sample(df1.query( "default==1").shape[0])])

df2008 = pd.concat([ df2.query( "default==1"),
                    df2.query( "default==0").sample(df2.query( "default==1").shape[0])])

df2009 = pd.concat([ df3.query( "default==1"),
                    df3.query( "default==0").sample(df3.query( "default==1").shape[0])])
df_total = pd.concat( [df2007, df2008, df2009])

In [ ]:
df_total2 = pd.get_dummies( df_total, columns=["year"]).head()

4. store-sales-time-series-forecasitng.zip
- train.csv -> id, date, store_nbr, familty, sales, onpomotion
- date->datetime 지정
- date->연도를 추출 year라는 컬럼
- pivot_table
 - 행: year, 열: family, 값: sales의 합
 - 시계열->연도별로 원하시는 컬럼으로 예측
 - 연도, 월 추출->"연도-월"->pivot_table

- train.csv와 stores.csv를 연결 merge(store_nbr) -> df
df1.merge(df2, left_on="컬럼", right_on="컬럼)

- stores.csv -> city, type, cluster->df에서50000개 샘플링
- 분기quarter, 요일(day_name(), city, type, cluster가 X변수, sales가 y변수
- sales->ML regression



In [35]:
!unzip store-sales-time-series-forecasting.zip

Archive:  store-sales-time-series-forecasting.zip
   creating: store-sales-time-series-forecasting/
  inflating: store-sales-time-series-forecasting/holidays_events.csv  
  inflating: store-sales-time-series-forecasting/oil.csv  
  inflating: store-sales-time-series-forecasting/sample_submission.csv  
  inflating: store-sales-time-series-forecasting/stores.csv  
  inflating: store-sales-time-series-forecasting/test.csv  
  inflating: store-sales-time-series-forecasting/train.csv  
  inflating: store-sales-time-series-forecasting/transactions.csv  


In [ ]:
import pandas as pd
df = pd.read_csv("data/train.csv")
df.shape

(3000888, 6)

In [ ]:
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [ ]:
df["date"] = pd.to_datetime( df.date)
df["year"] = df.date.dt.year

In [ ]:
df["month"] = df.date.dt.month
df["year2"] = df.year.astype("str").str.cat(df.month.astype(str), sep="-")

In [ ]:
df2 = df.pivot_table( index="year2", columns="family",
                     values="sales", aggfunc=sum)

/tmp/ipython-input-53-4226228238.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df2 = df.pivot_table( index="year2", columns="family",


In [ ]:
df2.shape

(56, 33)

In [ ]:
df2.AUTOMOTIVE

In [ ]:
import pandas as pd
df3 = pd.read_csv("data/train.csv")
df4 = pd.read_csv("data/stores.csv")

In [ ]:
df3 = df3.sample(50000)

In [ ]:
df5 = df3.merge(df4, left_on="store_nbr", right_on="store_nbr")

In [ ]:
df5.sales.hist(bins=50)

In [ ]:
df5["date"] = pd.to_datetime(df5.date)
df5["day"] = df5.date.dt.day_name()
df5["qt"] = df5.date.dt.quarter
df6 = df5[["qt", "day", "city", "type", "cluster"]]
df6["qt"] = df6["qt"].astype("object")
df6["cluster"] = df6["cluster"].astype("object")
df6_ohe = pd.get_dummies(df6)

In [ ]:
df6_ohe.shape

(50000, 55)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df6_ohe, df5.sales)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

np.float64(1186134.1420593634)

In [ ]:
(((rf.predict(X_test)-y_test)**2).mean())**0.5   #mse

np.float64(1089.0978569712472)

In [ ]:
df5.sales.describe()

5. home_default.zip
- 결측치 처리: 변수별로...
- target: TARGET,
- X:ohe
- 분류적용
- 테스트 성능 평가




In [38]:
!unzip home_default.zip

Archive:  home_default.zip
  inflating: home_default.csv        


In [ ]:
df = pd.read_csv("home_default.csv")
df.head()

In [ ]:
df.isnull().mean().sort_values(ascending=False).head(8)

In [ ]:
df.AMT_REQ_CREDIT_BUREAU_YEAR

In [ ]:
df.OWN_CAR_AGE.fillna(0)

In [43]:
df.AMT_REQ_CREDIT_BUREAU_YEAR
df2 = df.iloc[:, :28]
df2.drop("OWN_CAR_AGE", axis=1, inplace=True)


In [ ]:
#분류.....
pd.get_dummies(df2)

6. mortdefault모형에 대해서 fast API로 예측을 제공해보세요